## Import

In [10]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
import matplotlib.pyplot as plt

from scipy.stats import spearmanr
from scipy.stats import rankdata

In [2]:
output_path = "output_phenotypes"

## Load

In [3]:
input_path = "../input data/crca_xenium.h5ad"
adata = sc.read_h5ad(input_path)

In [5]:
adata_all_regions = adata.copy()
adata = adata[adata.obs['tissue_region'].isin(['core', 'margin'])].copy()

In [6]:
# normalize 
adata.X = adata.layers['norm'].copy()

In [7]:
gene_classifier = pd.read_excel("../input data/gene_classifier.xlsx", sheet_name="gene annotation and ranking")

In [8]:
genes = gene_classifier["SYMBOL"].tolist()

In [ ]:
# check how many genes are actually in our panel
genes_in_panel = [g for g in genes if g in adata.var_names]
genes_in_panel

In [ ]:
# just 11 out of 42 genes are in our panel ...
# maybe it is better to try computing our own signature

## Phenotype score

In [ ]:
# From the paper: assignments were based on the highest Spearman rank-correlations between the unknown samples and ranked expressions of classifier genes per spatial phenotypes of the discovery set

In [20]:
df = gene_classifier.set_index("SYMBOL")[["excl_rank", "infl_rank", "ign_rank"]]

In [21]:
# Filter for only common genes
common_genes = [g for g in df.index if g in adata.var_names]
df = df.loc[common_genes]

In [24]:
# recompute ranks on what is left (otherwise Spearman correlation with our patient's ranks does not make sense)
df_ranked = df.rank(method="dense", ascending=True)

In [25]:
# for Spearman we want highest values == more important
max_rank = df_ranked.max().max()
ref_scores = (max_rank - df_ranked + 1).astype(float) # max_rank - rank + 1

# no need to normalize because Sperman works with ranks

In [26]:
ref_scores

,excl_rank,infl_rank,ign_rank
SYMBOL,,,
CCL5,7.0,8.0,7.0
CEACAM6,4.0,1.0,4.0
CORO1A,8.0,7.0,8.0
CXCL13,6.0,6.0,6.0
GZMB,2.0,4.0,2.0
IGHG1,11.0,11.0,10.0
IL2RG,1.0,2.0,1.0
IL7R,9.0,9.0,9.0
LCK,5.0,5.0,5.0


In [ ]:
# here we can confirm that 11 genes are not enough
# we are losing a lot of the variability of the signatures, many genes have the same importance

In [17]:
# Compute mean expression per gene per patient

expr = adata[:, common_genes].X # shape: (n_cells, n_genes)
expr = expr.toarray() if hasattr(expr, "toarray") else expr

expr_df = pd.DataFrame(expr, index=adata.obs_names, columns=common_genes)
expr_df['patient_id'] = adata.obs['patient_id'].values

expr_by_patient = expr_df.groupby('patient_id')[common_genes].mean()

/tmp/ipykernel_561505/119148098.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  expr_by_patient = expr_df.groupby('patient_id')[common_genes].mean()


In [27]:
expr_by_patient

,CCL5,CEACAM6,CORO1A,CXCL13,GZMB,IGHG1,IL2RG,IL7R,LCK,NKG7,SDC1
patient_id,,,,,,,,,,,
a,0.047413,0.415135,0.070333,0.191157,0.047856,0.336998,0.205706,0.124869,0.031633,0.031886,0.778777
b,0.075619,0.968254,0.148845,0.049460,0.030996,0.368707,0.458232,0.131832,0.063396,0.069552,0.451521
c,0.040127,0.925729,0.101139,0.049327,0.049359,0.578116,0.418407,0.052455,0.029858,0.032293,0.518479
d,0.017030,0.750287,0.068534,0.022127,0.004925,0.522569,0.298651,0.068892,0.021591,0.012769,0.693040
e,0.025214,1.187447,0.048652,0.025718,0.014174,0.132480,0.203304,0.077159,0.025764,0.016367,0.458192
f,0.026622,0.762114,0.042832,0.029878,0.191461,0.396431,0.158380,0.061706,0.026159,0.013232,0.865979
g,0.007195,1.110180,0.107165,0.006124,0.007468,0.229432,0.350284,0.058307,0.175037,0.004391,0.533801
h,0.015604,1.237008,0.071555,0.017923,0.026187,0.209700,0.205966,0.104689,0.024640,0.010502,0.546511
i,0.014530,0.412131,0.057585,0.028655,0.049640,0.081809,0.258078,0.057370,0.014280,0.009732,0.755010


In [28]:
tie_delta = 0.03 # if top_corr - second_corr < tie_delta -> "ambiguous"

results = []
for pid, row in expr_by_patient.iterrows():
    
    # patient expression (genes ordered)
    patient_expr = row[ref_scores.index].values.astype(float) 

    # from expression to rank: rankdata 
    # use method 'average' to handle ties
    patient_rank_for_comp = (np.max(rankdata(patient_expr, method='average')) - rankdata(patient_expr, method='average') + 1) # for having higher expr -> higher rank

    # reference vector per phenotype (genes in same order)
    ref_excl = ref_scores["excl_rank"].values
    ref_infl = ref_scores["infl_rank"].values
    ref_ign  = ref_scores["ign_rank"].values

    # compute Spearman correlation between patient_rank_for_comp and ref vectors
    corr_excl, pval_excl = spearmanr(patient_rank_for_comp, ref_excl)
    corr_infl, pval_infl = spearmanr(patient_rank_for_comp, ref_infl)
    corr_ign,  pval_ign = spearmanr(patient_rank_for_comp, ref_ign)

    corrs = np.array([corr_excl, corr_infl, corr_ign], dtype=float)
    
    # decide assignment
    order = np.argsort(-corrs)  # descending order
    top = corrs[order[0]]
    second = corrs[order[1]]
    if (top - second) < tie_delta:
        assigned = "ambiguous"
    else:
        assigned = ["excluded","inflamed","ignored"][order[0]]

    results.append({
        'patient_id': pid,
        'corr_excluded': float(corr_excl),
        'pval_excluded': float(pval_excl),
        'corr_inflamed': float(corr_infl),
        'pval_inflamed': float(pval_infl),
        'corr_ignored': float(corr_ign),
        'pval_ignored': float(pval_excl),
        'assigned_phenotype': assigned
    })

results_df = pd.DataFrame(results).set_index('patient_id')

In [39]:
results_df

,corr_excluded,pval_excluded,corr_inflamed,pval_inflamed,corr_ignored,pval_ignored,assigned_phenotype
patient_id,,,,,,,
a,-0.336364,0.311824,-0.190909,0.573913,-0.354545,0.284693,inflamed
b,-0.200000,0.555445,0.009091,0.978837,-0.209091,0.537221,inflamed
c,-0.290909,0.385457,-0.118182,0.729285,-0.281818,0.401145,inflamed
d,-0.409091,0.211545,-0.172727,0.611542,-0.418182,0.200570,inflamed
e,-0.309091,0.355028,-0.081818,0.810990,-0.327273,0.325895,inflamed
f,-0.254545,0.450037,-0.172727,0.611542,-0.272727,0.417141,inflamed
g,-0.154545,0.650034,0.018182,0.957685,-0.172727,0.611542,inflamed
h,-0.281818,0.401145,-0.109091,0.749509,-0.290909,0.385457,inflamed
i,-0.263636,0.433441,-0.127273,0.709215,-0.290909,0.385457,inflamed


In [ ]:
# it is clear that defining the signature with only 11 genes is not so helpful

In [46]:
results_df.to_csv(os.path.join(output_path,"phenotype_assignment.csv"))